In [2]:
import os 
import torch 
import argparse
import numpy as np

from utils import *
from data.dataset import data_loader, data_loader_attacks

import mlp

def majority_voting(data_loader, model, mlps_list):
    """
    SEViT performance with majority voting. 

    Args: 
    data_loader: loader of test samples for clean images, or attackes generated from the test samples
    model: ViT model 
    mlps_list: list of intermediate MLPs

    Return: 
    Accuracy. 

    """
    acc_ = 0.0 
    for images, labels in data_loader:
        final_prediction = []
        images = images.cuda()
        vit_output = model(images)
        vit_predictions = torch.argmax(vit_output.detach().cpu(), dim=-1)
        final_prediction.append(vit_predictions.detach().cpu())
        x = model.patch_embed(images)
        x_0 = model.pos_drop(x)
        i=0
        for mlp in mlps_list:
            x_0 = model.blocks[i](x_0)
            mlp_output = mlp(x_0)
            mlp_predictions = torch.argmax(mlp_output.detach().cpu(), dim=-1)
            final_prediction.append(mlp_predictions.detach().cpu())
            i+=1
        stacked_tesnor = torch.stack(final_prediction,dim=1)
        preds_major = torch.argmax(torch.nn.functional.one_hot(stacked_tesnor).sum(dim=1), dim=-1)
        acc = (preds_major == labels).sum().item()/len(labels)
        acc_ += acc
    final_acc = acc_ / len(data_loader)
    print(f'Final Accuracy From Majority Voting = {(final_acc *100) :.3f}%' )
    return final_acc

def average(data_loader, model, mlps_list):
    """
    SEViT performance with averaging the outputs. 

    Args: 
    data_loader: loader of test samples for clean images, or attackes generated from the test samples
    model: ViT model 
    mlps_list: list of intermediate MLPs

    Return: 
    Accuracy. 

    """
    acc_ = 0.0 
    for images, labels in data_loader:
        final_prediction = []
        images = images.cuda()
        vit_output = model(images)
        vit_predictions = vit_output.detach().cpu()
        final_prediction.append(vit_predictions)
        x = model.patch_embed(images)
        x_0 = model.pos_drop(x)
        i=0
        for mlp in mlps_list:
            x_0 = model.blocks[i](x_0)
            mlp_output = mlp(x_0)
            mlp_predictions = mlp_output.detach().cpu()
            final_prediction.append(mlp_predictions)
            i+=1
        stacked_tensor = torch.stack(final_prediction,dim=1)
        preds_avg = torch.mean(stacked_tensor, dim=1)
        preds_final = torch.argmax(preds_avg, dim=-1)
        acc = (preds_final == labels).sum().item()/len(labels)
        acc_ += acc
    final_acc = acc_ / len(data_loader)
    print(f'Final Accuracy From Averaging = {(final_acc *100) :.3f}%' )
    return final_acc

def weighted_average(data_loader, model, mlps_list):
    """
    SEViT performance with weighted average. 

    Args: 
    data_loader: loader of test samples for clean images, or attackes generated from the test samples
    model: ViT model 
    mlps_list: list of intermediate MLPs

    Return: 
    Accuracy. 

    """
    acc_ = 0.0 
    for images, labels in data_loader:
        final_prediction = []
        images = images.cuda()
        vit_output = model(images)
        final_prediction.append(vit_output.detach().cpu())
        x = model.patch_embed(images)
        x_0 = model.pos_drop(x)
        i = 0
        weights = []
        for mlp in mlps_list:
            x_0 = model.blocks[i](x_0)
            mlp_output = mlp(x_0)
            final_prediction.append(mlp_output.detach().cpu())
            weight = torch.mean(torch.abs(mlp_output))
            weights.append(weight)
            i += 1
        weights.append(torch.mean(torch.abs(vit_output)))
        weights = torch.tensor(weights)
        weights = weights / weights.sum()
        final_prediction = torch.stack(final_prediction, dim=1)
        final_prediction = torch.sum(final_prediction.float() * weights.unsqueeze(-1), dim=1)
        final_predictions = torch.argmax(final_prediction, dim=-1)
        acc = (final_predictions == labels).sum().item()/len(labels)
        acc_ += acc
    final_acc = acc_ / len(data_loader)
    print(f'Final Accuracy From Weighted Average = {(final_acc *100) :.3f}%' )
    return final_acc

def countParams(model):
    total_params = sum(param.numel() for param in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Total Params: ", total_params)
    print("Trainable Params: ", trainable_params)

def vote(images_type, mlp_path):
    root_dir = 'data/TB_data/'
    vit_path = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'
    # mlp_path = 'models/MLP_new_chest/'

    model = torch.load(vit_path).cuda()
    model.eval()
    print('ViT is loaded!')

    MLPs_list = get_classifiers_list(MLP_path=mlp_path, num_classifiers=5)
    print('All MLPs are loaded!')

    if images_type == 'clean':
        image_folder_path = 'data/TB_data'
        loader_, dataset_ = data_loader(root_dir=image_folder_path, batch_size=15)
        majority_voting(data_loader=loader_['test'], model= model, mlps_list=MLPs_list)
        average(data_loader=loader_['test'], model= model, mlps_list=MLPs_list)
        weighted_average(data_loader=loader_['test'], model= model, mlps_list=MLPs_list)
    elif images_type == 'attack':
        image_folder_path = 'attack_images'
        attack_name = 'AUTOPGD'
        loader_, dataset_ = data_loader_attacks(root_dir=image_folder_path, attack_name= attack_name, batch_size=15)
        majority_voting(data_loader=loader_, model= model, mlps_list=MLPs_list)
        average(data_loader=loader_, model= model, mlps_list=MLPs_list)
        weighted_average(data_loader=loader_, model= model, mlps_list=MLPs_list)
    else:
        print('Input correct image type')

# path = 'ReVIT/models/cnns/'
# vote('attack', path)

# Before adversaring Training. Table 1-3

In [3]:
# Table 1: Clean Performance of MLP alternatives, from m=1 to m=5

def Table1(images_type, mlp_path):
    root_dir = 'data/TB_data/'
    vit_path = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'

    model = torch.load(vit_path).cuda()
    model.eval()
    print('ViT is loaded!')

    MLPs_list = get_classifiers_list(MLP_path=mlp_path, num_classifiers=5)
    print('All MLPs are loaded!')
    print("# of parameters of a module:")
    countParams(MLPs_list[0])

    if images_type == 'clean':
        image_folder_path = 'data/TB_data'
        loader_, dataset_ = data_loader(root_dir=image_folder_path, batch_size=15)
        for i in range(0, 6): # 0=VIT, 1 to 5 = mlp modules
            MLPs_subset = MLPs_list[:i]
            print(len(MLPs_subset))
            print(f"For m = {i}")        
            majority_voting(data_loader=loader_['test'], model= model, mlps_list=MLPs_subset)
            average(data_loader=loader_['test'], model= model, mlps_list=MLPs_subset)
            weighted_average(data_loader=loader_['test'], model= model, mlps_list=MLPs_subset)
    elif images_type == 'attack':
        image_folder_path = 'attack_images'
        attack_name = 'AUTOPGD'
        loader_, dataset_ = data_loader_attacks(root_dir=image_folder_path, attack_name= attack_name, batch_size=15)
        majority_voting(data_loader=loader_, model= model, mlps_list=MLPs_list)
        average(data_loader=loader_, model= model, mlps_list=MLPs_list)
        weighted_average(data_loader=loader_, model= model, mlps_list=MLPs_list)
    else:
        print('Input correct image type')

path_list = ['ReVIT/models/cnns', 'ReVIT/models/resnets_ft', 'ReVIT/models/resnets_tl', 'ReVIT/models/resnets_ft_cnn', 'ReVIT/models/resnets_tl_cnn', 'ReVIT/models/mlps']
for i in range(0, 6):
    path = path_list[i]
    print("For", path.split('/')[-1])
    Table1('clean', path)

# MLPs_list = get_classifiers_list(MLP_path=path, num_classifiers=5)
# print(MLPs_list)
# print('All MLPs are loaded!')

For cnns
ViT is loaded!
cnn_block_0_train_acc0.91_test_acc0.93.pth
MLP 1 is loaded!
cnn_block_1_train_acc0.94_test_acc0.95.pth
MLP 2 is loaded!
cnn_block_2_train_acc0.94_test_acc0.96.pth
MLP 3 is loaded!
cnn_block_3_train_acc0.95_test_acc0.96.pth
MLP 4 is loaded!
cnn_block_4_train_acc0.96_test_acc0.97.pth
MLP 5 is loaded!
cnn_block_5_train_acc0.96_test_acc0.96.pth
All MLPs are loaded!
# of parameters of a module:
Total Params:  1032770
Trainable Params:  1032770
['number of train images is 5670', 'number of valid images is 630', 'number of test images is 700']
Classes with index are: {'Normal': 0, 'Tuberculosis': 1}
['Normal', 'Tuberculosis']
0
For m = 0
Final Accuracy From Majority Voting = 96.377%
Final Accuracy From Averaging = 96.377%
Final Accuracy From Weighted Average = 96.377%
1
For m = 1
Final Accuracy From Majority Voting = 93.043%
Final Accuracy From Averaging = 95.507%
Final Accuracy From Weighted Average = 95.507%
2
For m = 2
Final Accuracy From Majority Voting = 96.232%
F

: 

In [ ]:
# Table 2: Attack Performance of MLP alternatives, from m=1 to m=4
# Perturbation Budget is 0.03

def Table2(images_type, mlp_path):
    root_dir = 'data/TB_data/'
    vit_path = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'

    model = torch.load(vit_path).cuda()
    model.eval()
    print('ViT is loaded!')

    MLPs_list = get_classifiers_list(MLP_path=mlp_path, num_classifiers=4)
    print('All MLPs are loaded!')
    print("# of parameters of a module:")
    countParams(MLPs_list[0])

    if images_type == 'attack':
        image_folder_path = 'attack_images'
        attack_list = ['FGSM', 'PGD', 'BIM', 'AUTOPGD', 'CW']
        for i in attack_list: #For all attack types in attack_list
            attack_name = i
            print(f'\nFor {attack_name} Attack samples:')
            loader_, dataset_ = data_loader_attacks(root_dir=image_folder_path, attack_name= attack_name, batch_size=15)
            for i in range(1, 5):
                MLPs_subset = MLPs_list[:i]
                print(len(MLPs_subset))
                print(f"For m = {i}")        
                majority_voting(data_loader=loader_, model= model, mlps_list=MLPs_subset)
                average(data_loader=loader_, model= model, mlps_list=MLPs_subset)
                weighted_average(data_loader=loader_, model= model, mlps_list=MLPs_subset)
    else:
        print('Input correct image type')

path_list = ['ReVIT/models/cnns', 'ReVIT/models/resnets_ft', 'ReVIT/models/resnets_tl', 'ReVIT/models/resnets_ft_cnn', 'ReVIT/models/resnets_tl_cnn', 'ReVIT/models/mlps']
for i in range(0, 6):
    path = path_list[i]
    print("\n\n-->>> For", path.split('/')[-1])
    Table2('attack', path)

# MLPs_list = get_classifiers_list(MLP_path=path, num_classifiers=5)
# print(MLPs_list)
# print('All MLPs are loaded!')

In [ ]:
# Table 2.1: Attack Performance (on adversarial examples generated for the surrogate model) of MLP alternatives, from m=1 to m=4
# Perturbation Budget is 0.03

def Table2_1(images_type, mlp_path):
    # root_dir = 'data/TB_data/'
    vit_path = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'

    model = torch.load(vit_path).cuda()
    model.eval()
    print('ViT is loaded!')

    MLPs_list = get_classifiers_list(MLP_path=mlp_path, num_classifiers=4)
    print('All MLPs are loaded!')
    print("# of parameters of a module:")
    countParams(MLPs_list[0])

    if images_type == 'attack':
        image_folder_path = 'attack_images_surrogate_0.03'
        attack_list = ['FGSM', 'PGD', 'BIM', 'AUTOPGD', 'CW']
        for i in attack_list: #For all attack types in attack_list
            attack_name = i
            print(f'\nFor {attack_name} Attack samples:')
            loader_, dataset_ = data_loader_attacks(root_dir=image_folder_path, attack_name= attack_name, batch_size=15)
            for i in range(1, 5):
                MLPs_subset = MLPs_list[:i]
                print(len(MLPs_subset))
                print(f"For m = {i}")        
                majority_voting(data_loader=loader_, model= model, mlps_list=MLPs_subset)
                average(data_loader=loader_, model= model, mlps_list=MLPs_subset)
                weighted_average(data_loader=loader_, model= model, mlps_list=MLPs_subset)
    else:
        print('Input correct image type')

path_list = ['ReVIT/models/cnns', 'ReVIT/models/resnets_ft', 'ReVIT/models/resnets_tl', 'ReVIT/models/resnets_ft_cnn', 'ReVIT/models/resnets_tl_cnn', 'ReVIT/models/mlps']
for i in range(0, 6):
    path = path_list[i]
    print("\n\n-->>> For", path.split('/')[-1])
    Table2_1('attack', path)

# MLPs_list = get_classifiers_list(MLP_path=path, num_classifiers=5)
# print(MLPs_list)
# print('All MLPs are loaded!')

In [ ]:
# Table 3: Performance of Surrogate model on Clean test and Attack Samples

# After Adversarial Training
The samples used for adversarial trainig are original train samples + fgsm attack samples

In [4]:
# Table 4: Clean Sample Performance of adversarially-trained MLP alternatives, after adversarial training, from m=1 to m=5

def Table4(images_type, mlp_path):
    # root_dir = 'data/TB_data/'
    vit_path = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'

    model = torch.load(vit_path).cuda()
    model.eval()
    print('ViT is loaded!')

    MLPs_list = get_classifiers_list(MLP_path=mlp_path, num_classifiers=5)
    print('All MLPs are loaded!')
    print("# of parameters of a module:")
    countParams(MLPs_list[0])

    if images_type == 'clean':
        print("Testing on Original Clean Test dataset") # Note, Original test data, and adversarial test data are same, since adversarial samples are used just for training
        image_folder_path = 'adv_data/TB_adversarial_data'
        loader_, dataset_ = data_loader(root_dir=image_folder_path, batch_size=15)
        for i in range(1, 6):
            MLPs_subset = MLPs_list[:i]
            print(len(MLPs_subset))
            print(f"For m = {i}")        
            majority_voting(data_loader=loader_['test'], model= model, mlps_list=MLPs_subset)
            average(data_loader=loader_['test'], model= model, mlps_list=MLPs_subset)
            weighted_average(data_loader=loader_['test'], model= model, mlps_list=MLPs_subset)
    # elif images_type == 'attack':
    #     image_folder_path = 'attack_images'
    #     attack_name = 'AUTOPGD'
    #     loader_, dataset_ = data_loader_attacks(root_dir=image_folder_path, attack_name= attack_name, batch_size=15)
    #     majority_voting(data_loader=loader_, model= model, mlps_list=MLPs_list)
    #     average(data_loader=loader_, model= model, mlps_list=MLPs_list)
    #     weighted_average(data_loader=loader_, model= model, mlps_list=MLPs_list)
    else:
        print('Input correct image type')

path_list = ['ReVIT/adv_train_models/cnns', 'ReVIT/adv_train_models/resnets_ft', 'ReVIT/adv_train_models/resnets_tl', 'ReVIT/adv_train_models/resnets_ft_cnn', 'ReVIT/adv_train_models/resnets_tl_cnn', 'ReVIT/adv_train_models/mlps']
for i in range(0, 6):
    path = path_list[i]
    print("\n\n-->>For", path.split('/')[-1])
    Table4('clean', path)

# MLPs_list = get_classifiers_list(MLP_path=path, num_classifiers=5)
# print(MLPs_list)
# print('All MLPs are loaded!')



-->>For cnns
ViT is loaded!
cnn_block_0_train_acc0.91_test_acc0.95.pth
MLP 1 is loaded!
cnn_block_1_train_acc0.94_test_acc0.96.pth
MLP 2 is loaded!
cnn_block_2_train_acc0.94_test_acc0.97.pth
MLP 3 is loaded!
cnn_block_3_train_acc0.95_test_acc0.97.pth
MLP 4 is loaded!
cnn_block_4_train_acc0.95_test_acc0.97.pth
MLP 5 is loaded!
cnn_block_5_train_acc0.95_test_acc0.97.pth
All MLPs are loaded!
# of parameters of a module:
Total Params:  1032770
Trainable Params:  1032770
Testing on Original Clean Test dataset
['number of train images is 6360', 'number of valid images is 630', 'number of test images is 700']
Classes with index are: {'Normal': 0, 'Tuberculosis': 1}
['Normal', 'Tuberculosis']
1
For m = 1
Final Accuracy From Majority Voting = 95.507%
Final Accuracy From Averaging = 96.667%
Final Accuracy From Weighted Average = 96.522%
2
For m = 2
Final Accuracy From Majority Voting = 95.652%
Final Accuracy From Averaging = 96.957%
Final Accuracy From Weighted Average = 96.667%
3
For m = 3
Fi

In [2]:
# Table 5: Attack Performance of adversarially-trained (on FGSM) MLP alternatives, after adversarial training, from m=1 to m=4
# Perturbation Budget is 0.03

def Table5(images_type, mlp_path):
    # root_dir = 'data/TB_data/'
    vit_path = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'

    model = torch.load(vit_path).cuda()
    model.eval()
    print('ViT is loaded!')

    MLPs_list = get_classifiers_list(MLP_path=mlp_path, num_classifiers=4)
    print('All MLPs are loaded!')
    print("# of parameters of a module:")
    countParams(MLPs_list[0])

    if images_type == 'attack':
        image_folder_path = 'attack_images'
        attack_list = ['FGSM', 'PGD', 'BIM', 'AUTOPGD', 'CW']
        for i in attack_list: #For all attack types in attack_list
            attack_name = i
            print(f'\nFor {attack_name} Attack samples:')
            loader_, dataset_ = data_loader_attacks(root_dir=image_folder_path, attack_name= attack_name, batch_size=15)
            for i in range(1, 5):
                MLPs_subset = MLPs_list[:i]
                print(len(MLPs_subset))
                print(f"For m = {i}")        
                majority_voting(data_loader=loader_, model= model, mlps_list=MLPs_subset)
                average(data_loader=loader_, model= model, mlps_list=MLPs_subset)
                weighted_average(data_loader=loader_, model= model, mlps_list=MLPs_subset)
    else:
        print('Input correct image type')

path_list = ['ReVIT/adv_train_models/cnns', 'ReVIT/adv_train_models/resnets_ft', 'ReVIT/adv_train_models/resnets_tl', 'ReVIT/adv_train_models/resnets_ft_cnn', 'ReVIT/adv_train_models/resnets_tl_cnn', 'ReVIT/adv_train_models/mlps']
for i in range(0, 6):
    path = path_list[i]
    print("\n\n-->>> For", path.split('/')[-1])
    Table5('attack', path)

# MLPs_list = get_classifiers_list(MLP_path=path, num_classifiers=5)
# print(MLPs_list)
# print('All MLPs are loaded!')



-->>> For cnns
ViT is loaded!
cnn_block_0_train_acc0.91_test_acc0.95.pth
MLP 1 is loaded!
cnn_block_1_train_acc0.94_test_acc0.96.pth
MLP 2 is loaded!
cnn_block_2_train_acc0.94_test_acc0.97.pth
MLP 3 is loaded!
cnn_block_3_train_acc0.95_test_acc0.97.pth
MLP 4 is loaded!
cnn_block_4_train_acc0.95_test_acc0.97.pth
All MLPs are loaded!
# of parameters of a module:
Total Params:  1032770
Trainable Params:  1032770

For FGSM Attack samples:
number of images is 690
Classes with index are: {'Normal': 0, 'Tuberculosis': 1}
1
For m = 1
Final Accuracy From Majority Voting = 71.304%
Final Accuracy From Averaging = 77.246%
Final Accuracy From Weighted Average = 72.464%
2
For m = 2
Final Accuracy From Majority Voting = 87.826%
Final Accuracy From Averaging = 83.188%
Final Accuracy From Weighted Average = 82.609%
3
For m = 3
Final Accuracy From Majority Voting = 89.420%
Final Accuracy From Averaging = 87.971%
Final Accuracy From Weighted Average = 87.246%
4
For m = 4
Final Accuracy From Majority Vo

KeyboardInterrupt: 

: 

In [2]:
# Table 5_1: Attack Performance (on adversarial examples generated for the surrogate model) of adversarially-trained (on FGSM) MLP alternatives, 
# after adversarial training, from m=1 to m=4
# Perturbation Budget is 0.03

def Table5_1(images_type, mlp_path):
    # root_dir = 'data/TB_data/'
    vit_path = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'

    model = torch.load(vit_path).cuda()
    model.eval()
    print('ViT is loaded!')

    MLPs_list = get_classifiers_list(MLP_path=mlp_path, num_classifiers=4)
    print('All MLPs are loaded!')
    print("# of parameters of a module:")
    countParams(MLPs_list[0])

    if images_type == 'attack':
        image_folder_path = 'attack_images_surrogate_0.03'
        attack_list = ['FGSM', 'PGD', 'BIM', 'AUTOPGD', 'CW']
        for i in attack_list: #For all attack types in attack_list
            attack_name = i
            print(f'\nFor {attack_name} Attack samples:')
            loader_, dataset_ = data_loader_attacks(root_dir=image_folder_path, attack_name= attack_name, batch_size=15)
            for i in range(1, 5):
                MLPs_subset = MLPs_list[:i]
                print(len(MLPs_subset))
                print(f"For m = {i}")        
                majority_voting(data_loader=loader_, model= model, mlps_list=MLPs_subset)
                average(data_loader=loader_, model= model, mlps_list=MLPs_subset)
                weighted_average(data_loader=loader_, model= model, mlps_list=MLPs_subset)
    else:
        print('Input correct image type')

path_list = ['ReVIT/adv_train_models/cnns', 'ReVIT/adv_train_models/resnets_ft', 'ReVIT/adv_train_models/resnets_tl', 'ReVIT/adv_train_models/resnets_ft_cnn', 'ReVIT/adv_train_models/resnets_tl_cnn', 'ReVIT/adv_train_models/mlps']
for i in range(0, 6):
    path = path_list[i]
    print("\n\n-->>> For", path.split('/')[-1])
    Table5_1('attack', path)

# MLPs_list = get_classifiers_list(MLP_path=path, num_classifiers=5)
# print(MLPs_list)
# print('All MLPs are loaded!')



-->>> For cnns
ViT is loaded!
cnn_block_0_train_acc0.91_test_acc0.95.pth
MLP 1 is loaded!
cnn_block_1_train_acc0.94_test_acc0.96.pth
MLP 2 is loaded!
cnn_block_2_train_acc0.94_test_acc0.97.pth
MLP 3 is loaded!
cnn_block_3_train_acc0.95_test_acc0.97.pth
MLP 4 is loaded!
cnn_block_4_train_acc0.95_test_acc0.97.pth
All MLPs are loaded!
# of parameters of a module:
Total Params:  1032770
Trainable Params:  1032770

For FGSM Attack samples:
number of images is 690
Classes with index are: {'Normal': 0, 'Tuberculosis': 1}
1
For m = 1
Final Accuracy From Majority Voting = 92.464%
Final Accuracy From Averaging = 94.493%
Final Accuracy From Weighted Average = 94.348%
2
For m = 2
Final Accuracy From Majority Voting = 94.203%
Final Accuracy From Averaging = 95.217%
Final Accuracy From Weighted Average = 95.072%
3
For m = 3
Final Accuracy From Majority Voting = 94.928%
Final Accuracy From Averaging = 94.783%
Final Accuracy From Weighted Average = 94.783%
4
For m = 4
Final Accuracy From Majority Vo

In [4]:
import sys

with open('Table5.txt', 'w') as f:
    # Redirect stdout to the file
    sys.stdout = f
    
    path = 'ReVIT/models/cnns/'
    vote('attack', path)
    
    # Restore stdout
    sys.stdout = sys.__stdout__